<a href="https://colab.research.google.com/github/Sep-eg/kaggle_survey2021/blob/main/2021_kaggle_survey_GHK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 불러오기

In [ ]:
!mkdir ~/.kaggle # api키 보관용 폴더 생성
!cp kaggle.json ~/.kaggle # api키 파일을 보관용 폴더에 복사
!kaggle competitions download -c kaggle-survey-2021 # kaggle survey 2021 데이터 다운로드

In [ ]:
import pandas as pd

In [ ]:
response = pd.read_csv("/content/kaggle_survey_2021_responses.csv.zip")

In [ ]:
response.head()

In [ ]:
response = response.loc[:][1:] # 질문을 담고있는 첫번째 행 제거
response.reset_index(drop=True, inplace=True) # 인덱스 초기화

In [ ]:
response.head()

## 질문합치기
모두를 대상으로 질문한 총 42개의 메인 문항과

비전문가 계열에게 질문한 8개의 보충 문항을 분류한뒤 각각을 질문 카테고리별로 합칩니다.

In [ ]:
questions = response.columns

In [ ]:
# 보충 질문(비전문가 문항)과 메인 질문 구분
supple_survey_columns = [] # 보충질문의 칼럼명을 담기위한 리스트

# 모든 질문을 순회하면서 보충 질문을 담은 칼럼명을 저장
for quest in questions:
    if '_' in quest:
        if quest.split('_')[1] == 'B': # B가 질문에 포함되면 리스트에 추가
            supple_survey_columns.append(quest)
    else:
        pass

In [ ]:
# 보충 질문을 제외한 모든 문항은 메인 질문으로 저장
main_survey_columns = questions.drop(supple_survey_columns)

많은 질문이 다중선택 질문이며, 모든 답변이 각각의 칼럼으로 나누어져 있습니다.

이를 합치기 위해 dictionary를 활용하여 각 질문 번호를 key, 답변 번호를 리스트로 이루어진 value로 저장하겠습니다.

In [ ]:
supple_dict = {col.split('_')[0] : [] for col in supple_survey_columns}
for col in supple_survey_columns:
    supple_dict[col.split('_')[0]].append(col)

main_dict = {col.split('_')[0]: [] for col in main_survey_columns}
for col in main_survey_columns:
    main_dict[col.split('_')[0]].append(col)

답변을 하나로 합치기 위해 더할 때 Nan값은 더해질 수 없습니다.(무시하고 더할 경우 모든 값이 Nan)

해당 데이터의 결측치는 모두 설문 미응답이므로 'Not responded'로 대체하겠습니다.

In [ ]:
supple_response = response[supple_survey_columns].fillna('Not responded')
main_response = response[main_survey_columns].fillna('Not responded')

메인 답변과 보충 답변을 나누는 기반작업은 끝났고, 본격적으로 답변을 합쳐 하나의 칼럼에 담도록 하겠습니다.

In [ ]:
def split_dropnone(response): 
    '''
    더해진 답변들을 리스트로 만들기 위한 함수
    '''
    # 각각의 답변을 순회하면서 Not responded가 아니라면, tmp_list에 추가
    tmp_list = [data for data in response.split(',') if data != 'Not responded']
    if len(tmp_list) == 0: # tmp_list의 길이가 0이라면 즉, 답변이 없었다면 Not responded 반환
        return 'Not responded'
    else: # tmp_list가 비어있지 않다면 tmp_list 반환
        return tmp_list

In [ ]:
for key, value in supple_dict.items():
    # 답변을 합치기 위해 첫번째 답변을 tmp_series에 담고 나머지 답변 순회
    tmp_series = pd.Series(supple_response[value[0]])
    for col in value[1:]:
        tmp_series = tmp_series + ',' + supple_response[col] # ,를 기준으로 답변취합
    supple_response[key] = tmp_series # 질문번호를 칼럼명으로 취합된 답변을 담은 새로운 칼럼 생성
    supple_response.drop(value, axis=1, inplace=True) # 기존의 나눠져있는 답변칼럼 모두 제거
    supple_response[key] = supple_response[key].map(split_dropnone) # ,를 기준으로 나눠진 답변을 리스트로 변경

In [ ]:
# 기본적인 원리는 보충질문과 같지만, 단일 답변으로 구성된 질문이 존재
for key, value in main_dict.items():
    tmp_series = pd.Series(main_response[value[0]])
    flag = 0 # 단일답변 질문을 골라내기 위해 flag변수 설정
    if len(value) > 1: # 다중선택 질문일 경우 실행
        flag += 1
        for col in value[1:]:
            tmp_series = tmp_series + ',' + main_response[col]
        main_response[key] = tmp_series
    if flag == 1: # 다중선택 질문일 경우 나머지 칼럼 제거 및 답변을 리스트로 저장
        main_response.drop(value, axis=1, inplace=True)
        main_response[key] = main_response[key].map(split_dropnone)